In [22]:
from read_user_timeline import read_user_timeline
import pickle
import os
import pandas as pd

In [15]:
office = ['@Microsoft365','@MSFTDynamics365','@linkedin']
computing = ['@Xbox','@surface','@windows']
cloud = ['@Azure','@SQLServer','@windowsserver','@github']

In [16]:
if(os.path.isfile('@Microsoft365tweets.sav')==False):
    print('hello')

In [25]:
tweets_office=[]
for thandle in office:
    if(os.path.isfile(thandle+'tweets.sav')==False):
        print('Extracting timeline for ' +thandle)
        read_user_timeline(thandle)
    else:
        print('Loading previously extracted timeline for ' + thandle)
        tweets_office.append(pickle.load(open(thandle+'tweets.sav','rb')))

Loading previously extracted timeline for @Microsoft365
Loading previously extracted timeline for @MSFTDynamics365
Loading previously extracted timeline for @linkedin


In [26]:
date_range = []
for i in range(len(tweets_office)):
    dates = [t['created_at'] for t in tweets_office[i]]
    dates = pd.to_datetime(dates)
    tmp=[]
    tmp.append(dates.min())
    tmp.append(dates.max())
    date_range.append(tmp)

In [27]:
date_range

[[Timestamp('2020-02-19 19:41:22+0000', tz='UTC'),
  Timestamp('2020-09-16 15:30:08+0000', tz='UTC')],
 [Timestamp('2019-08-20 15:00:02+0000', tz='UTC'),
  Timestamp('2020-09-16 13:30:01+0000', tz='UTC')],
 [Timestamp('2018-02-21 18:12:00+0000', tz='UTC'),
  Timestamp('2020-09-16 13:15:00+0000', tz='UTC')]]